In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import xml.etree.ElementTree as et

In [2]:
train_df = pd.read_csv('train.csv')
xtree = et.parse('holidays.xml')
weather_df = pd.read_csv('weather-sfcsv.csv')

In [3]:
def extract_date(df, column_name, date_format):
    df['timestamp_datetime'] = [datetime.strptime(d.split('.')[0], date_format) for d in df[column_name]]
    df['Year'] = [d.year for d in df['timestamp_datetime']]
    df['Month'] = [d.month for d in df['timestamp_datetime']]
    df['Day'] = [d.day for d in df['timestamp_datetime']]
    if len(date_format) > 8:
        df['Hour'] = [d.hour for d in df['timestamp_datetime']]
    df.drop([column_name, 'timestamp_datetime'], axis= 1, inplace=True)

In [4]:
xroot = xtree.getroot() 
df_cols = ['date', 'description']
rows = []
for node in xroot: 
    s_name = node.find("date").text if node is not None else None
    s_mail = node.find("description").text if node is not None else None
    rows.append({"date": s_name, "description": s_mail})
holiday_df = pd.DataFrame(rows, columns = df_cols)
extract_date(holiday_df, 'date', '%Y-%m-%d')

In [5]:
print("Training Data info:")
print(train_df.info())
print(train_df.head())
print("\nWeather Data info:")
print(weather_df.info())
print(weather_df.head())
print("\nHoliday Data info:")
print(holiday_df.info())
print(holiday_df.head())

Training Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6407 entries, 0 to 6406
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            6407 non-null   int64  
 1   Lat           6407 non-null   float64
 2   Lng           6407 non-null   float64
 3   Bump          6407 non-null   bool   
 4   Distance(mi)  6407 non-null   float64
 5   Crossing      6407 non-null   bool   
 6   Give_Way      6407 non-null   bool   
 7   Junction      6407 non-null   bool   
 8   No_Exit       6407 non-null   bool   
 9   Railway       6407 non-null   bool   
 10  Roundabout    6407 non-null   bool   
 11  Stop          6407 non-null   bool   
 12  Amenity       6407 non-null   bool   
 13  Side          6407 non-null   object 
 14  Severity      6407 non-null   int64  
 15  timestamp     6407 non-null   object 
dtypes: bool(9), float64(3), int64(2), object(2)
memory usage: 406.8+ KB
None
   ID        Lat     

In [6]:
extract_date(train_df, 'timestamp', '%Y-%m-%d %H:%M:%S')
train_df = train_df.drop(columns=['ID'],axis=1)
train_df.head()

,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,Year,Month,Day,Hour
0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016,3,25,15
1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020,5,5,19
2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016,9,16,19
3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020,3,29,19
4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019,10,9,8


In [7]:
train_df_wd = train_df.drop_duplicates(subset=train_df.keys(), keep='last')
weather_df_wd = weather_df.drop_duplicates(subset=['Year', 'Day', 'Month', 'Hour'], keep='last')

In [49]:
df = pd.merge(train_df_wd, weather_df_wd,  how='left', left_on=['Year','Day', 'Month', 'Hour'], right_on = ['Year','Day', 'Month', 'Hour'])
df.sort_values(by=['Year','Month', 'Day'], inplace=True)
df.head()

,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,...,Day,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
3571,37.807910,-122.367400,False,0.000,False,False,True,False,False,False,...,23,20,Scattered Clouds,NaN,NaN,59.0,67.0,12.7,10.0,No
3637,37.779530,-122.400124,False,0.010,False,False,False,False,False,False,...,23,9,Mostly Cloudy,NaN,NaN,55.0,64.0,NaN,10.0,No
5912,37.745080,-122.404930,False,0.150,False,False,True,False,False,False,...,23,9,Mostly Cloudy,NaN,NaN,55.0,64.0,NaN,10.0,No
225,37.791954,-122.469727,False,0.010,False,False,False,False,False,False,...,24,18,Scattered Clouds,NaN,NaN,57.9,87.0,21.9,10.0,No
955,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,...,24,8,Partly Cloudy,NaN,NaN,55.0,80.0,6.9,10.0,No


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6295 entries, 3571 to 6263
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Lat                6295 non-null   float64
 1   Lng                6295 non-null   float64
 2   Bump               6295 non-null   bool   
 3   Distance(mi)       6295 non-null   float64
 4   Crossing           6295 non-null   bool   
 5   Give_Way           6295 non-null   bool   
 6   Junction           6295 non-null   bool   
 7   No_Exit            6295 non-null   bool   
 8   Railway            6295 non-null   bool   
 9   Roundabout         6295 non-null   bool   
 10  Stop               6295 non-null   bool   
 11  Amenity            6295 non-null   bool   
 12  Side               6295 non-null   object 
 13  Severity           6295 non-null   int64  
 14  Year               6295 non-null   int64  
 15  Month              6295 non-null   int64  
 16  Day                62

In [51]:
df.isnull().sum()

Lat                     0
Lng                     0
Bump                    0
Distance(mi)            0
Crossing                0
Give_Way                0
Junction                0
No_Exit                 0
Railway                 0
Roundabout              0
Stop                    0
Amenity                 0
Side                    0
Severity                0
Year                    0
Month                   0
Day                     0
Hour                    0
Weather_Condition       1
Wind_Chill(F)        3118
Precipitation(in)    2871
Temperature(F)          2
Humidity(%)             2
Wind_Speed(mph)       306
Visibility(mi)          1
Selected                0
dtype: int64

In [52]:
df.dropna(axis=0, subset=['Weather_Condition', 'Temperature(F)', 'Humidity(%)', 'Visibility(mi)'], inplace=True)
df.isnull().sum()

Lat                     0
Lng                     0
Bump                    0
Distance(mi)            0
Crossing                0
Give_Way                0
Junction                0
No_Exit                 0
Railway                 0
Roundabout              0
Stop                    0
Amenity                 0
Side                    0
Severity                0
Year                    0
Month                   0
Day                     0
Hour                    0
Weather_Condition       0
Wind_Chill(F)        3116
Precipitation(in)    2870
Temperature(F)          0
Humidity(%)             0
Wind_Speed(mph)       305
Visibility(mi)          0
Selected                0
dtype: int64

In [53]:
df['Wind_Chill(F)'].interpolate(method='linear', inplace=True)
df['Precipitation(in)'].interpolate(method='linear', inplace=True)
df['Wind_Speed(mph)'].interpolate(method='linear', inplace=True)
df.isnull().sum()

Lat                    0
Lng                    0
Bump                   0
Distance(mi)           0
Crossing               0
Give_Way               0
Junction               0
No_Exit                0
Railway                0
Roundabout             0
Stop                   0
Amenity                0
Side                   0
Severity               0
Year                   0
Month                  0
Day                    0
Hour                   0
Weather_Condition      0
Wind_Chill(F)        678
Precipitation(in)     46
Temperature(F)         0
Humidity(%)            0
Wind_Speed(mph)        0
Visibility(mi)         0
Selected               0
dtype: int64

In [54]:
df.fillna(value=df.mean(), inplace=True)
df.isnull().sum()

<ipython-input-54-4d2cb040dcd5>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(value=df.mean(), inplace=True)


Lat                  0
Lng                  0
Bump                 0
Distance(mi)         0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Stop                 0
Amenity              0
Side                 0
Severity             0
Year                 0
Month                0
Day                  0
Hour                 0
Weather_Condition    0
Wind_Chill(F)        0
Precipitation(in)    0
Temperature(F)       0
Humidity(%)          0
Wind_Speed(mph)      0
Visibility(mi)       0
Selected             0
dtype: int64

In [55]:
df.replace({True: 1, False: 0, 'Yes': 1, 'No': 0, 'R': 1, 'L': 0}, inplace=True)

In [56]:
weather_conditions = df['Weather_Condition'].unique()

In [57]:
df.iloc[:4,10:]

,Stop,Amenity,Side,Severity,Year,Month,Day,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
3571,0,0,1,2,2016,3,23,20,Scattered Clouds,51.742547,0.013835,59.0,67.0,12.700000,10.0,0
3637,0,0,1,3,2016,3,23,9,Mostly Cloudy,51.742547,0.013835,55.0,64.0,15.766667,10.0,0
5912,0,0,1,2,2016,3,23,9,Mostly Cloudy,51.742547,0.013835,55.0,64.0,18.833333,10.0,0
225,0,0,1,2,2016,3,24,18,Scattered Clouds,51.742547,0.013835,57.9,87.0,21.900000,10.0,0


In [58]:
df.corr()

,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,...,Month,Day,Hour,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
Lat,1.000000,0.386898,NaN,0.036637,0.042491,0.011435,0.003886,1.052640e-02,-0.002532,NaN,...,-0.031627,0.007884,0.021232,2.920714e-02,-0.043418,0.041457,0.009523,-0.047333,0.001478,0.017103
Lng,0.386898,1.000000,NaN,0.072426,-0.122937,-0.032874,0.183286,-7.027928e-04,-0.031049,NaN,...,-0.018109,-0.004642,0.008503,-1.174283e-03,-0.007897,0.025421,0.015522,-0.064909,-0.026506,0.016744
Bump,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Distance(mi),0.036637,0.072426,NaN,1.000000,-0.019238,-0.007191,-0.029824,3.777838e-04,-0.033722,NaN,...,0.077305,0.008932,0.004400,-1.862853e-02,0.000033,-0.022384,0.002025,-0.020053,-0.000227,-0.006052
Crossing,0.042491,-0.122937,NaN,-0.019238,1.000000,0.072388,-0.159816,-3.803406e-03,0.429768,NaN,...,-0.006890,0.025010,-0.037193,-1.899235e-03,-0.016298,-0.040101,0.013822,-0.040813,0.014313,-0.005379
Give_Way,0.011435,-0.032874,NaN,-0.007191,0.072388,1.000000,-0.012346,-2.753220e-04,0.041403,NaN,...,0.011650,-0.005033,0.008091,2.000047e-02,-0.004102,0.017330,-0.024758,-0.013190,0.007411,-0.000389
Junction,0.003886,0.183286,NaN,-0.029824,-0.159816,-0.012346,1.000000,-7.126634e-03,-0.093908,NaN,...,-0.036197,-0.019051,0.017255,7.458918e-02,-0.016431,0.013706,-0.032847,-0.002442,-0.006794,0.010731
No_Exit,0.010526,-0.000703,NaN,0.000378,-0.003803,-0.000275,-0.007127,1.000000e+00,-0.002094,NaN,...,-0.006206,-0.015405,0.000334,-9.869186e-17,-0.002031,0.004806,0.003006,0.005985,0.004278,-0.000225
Railway,-0.002532,-0.031049,NaN,-0.033722,0.429768,0.041403,-0.093908,-2.094264e-03,1.000000,NaN,...,0.021890,0.023491,-0.013697,3.627148e-02,-0.020192,-0.005200,-0.021023,-0.009273,0.006953,-0.002962
Roundabout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df.drop(['Roundabout', 'Bump', 'Day'], axis=1, inplace=True)

In [60]:
df.iloc[:,10:].corr()

,Side,Severity,Year,Month,Hour,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
Side,1.000000,0.061376,-0.016249,-0.003547,0.029646,0.016840,0.000686,0.031358,-0.008165,0.016823,-0.010230,0.004147
Severity,0.061376,1.000000,-0.171061,-0.053281,-0.019082,-0.170256,0.055100,-0.024845,0.085291,0.015920,-0.014778,0.024455
Year,-0.016249,-0.171061,1.000000,-0.085373,0.040108,0.501869,-0.184226,-0.074050,-0.076163,-0.102577,-0.051944,0.007963
Month,-0.003547,-0.053281,-0.085373,1.000000,0.005463,0.204394,0.067150,0.155753,-0.067141,-0.085073,0.017105,-0.023741
Hour,0.029646,-0.019082,0.040108,0.005463,1.000000,0.121483,-0.024596,0.229191,-0.240791,0.237269,0.101258,-0.008730
Wind_Chill(F),0.016840,-0.170256,0.501869,0.204394,0.121483,1.000000,-0.308310,0.555980,-0.380073,0.081543,0.114440,-0.023790
Precipitation(in),0.000686,0.055100,-0.184226,0.067150,-0.024596,-0.308310,1.000000,-0.185325,0.098800,-0.014546,-0.338384,-0.001919
Temperature(F),0.031358,-0.024845,-0.074050,0.155753,0.229191,0.555980,-0.185325,1.000000,-0.592714,0.294927,0.162112,-0.017704
Humidity(%),-0.008165,0.085291,-0.076163,-0.067141,-0.240791,-0.380073,0.098800,-0.592714,1.000000,-0.182309,-0.309088,-0.030802
Wind_Speed(mph),0.016823,0.015920,-0.102577,-0.085073,0.237269,0.081543,-0.014546,0.294927,-0.182309,1.000000,0.047798,0.005423


In [61]:
df.head()

,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,...,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
3571,37.807910,-122.367400,0.000,0,0,1,0,0,0,0,...,3,20,Scattered Clouds,51.742547,0.013835,59.0,67.0,12.700000,10.0,0
3637,37.779530,-122.400124,0.010,0,0,0,0,0,0,0,...,3,9,Mostly Cloudy,51.742547,0.013835,55.0,64.0,15.766667,10.0,0
5912,37.745080,-122.404930,0.150,0,0,1,0,0,0,0,...,3,9,Mostly Cloudy,51.742547,0.013835,55.0,64.0,18.833333,10.0,0
225,37.791954,-122.469727,0.010,0,0,0,0,0,0,0,...,3,18,Scattered Clouds,51.742547,0.013835,57.9,87.0,21.900000,10.0,0
955,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,...,3,8,Partly Cloudy,51.742547,0.013835,55.0,80.0,6.900000,10.0,0


In [62]:
def normalize_data(df, *args):
    for arg in args:
        df[arg] = (df[arg] - df[arg].mean())/df[arg].std()

In [63]:
normalize_data(df, 'Lat', 'Lng', 'Wind_Speed(mph)', 'Wind_Chill(F)', 'Temperature(F)', 'Visibility(mi)')

In [64]:
df.describe()

,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,...,Year,Month,Hour,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
count,6.293000e+03,6.293000e+03,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,...,6293.000000,6293.000000,6293.000000,6.293000e+03,6293.000000,6.293000e+03,6293.000000,6.293000e+03,6.293000e+03,6293.000000
mean,-3.349137e-14,1.807667e-13,0.134155,0.083426,0.000477,0.242174,0.000159,0.026855,0.098522,0.037343,...,2018.387097,6.759733,12.846178,5.780993e-16,0.013835,-7.000421e-16,68.119021,4.516401e-17,-1.117809e-16,0.000318
std,1.000000e+00,1.000000e+00,0.395302,0.276547,0.021830,0.428433,0.012606,0.161673,0.298043,0.189616,...,1.372525,3.574934,5.813688,1.000000e+00,0.035632,1.000000e+00,16.278361,1.000000e+00,1.000000e+00,0.017826
min,-4.789971e+00,-3.693206e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2016.000000,1.000000,0.000000,-1.858416e+00,0.000000,-2.991521e+00,10.000000,-1.665298e+00,-5.644539e+00,0.000000
25%,-8.845452e-01,-2.177104e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2017.000000,4.000000,8.000000,-8.794668e-01,0.000000,-7.430253e-01,59.000000,-7.658764e-01,3.393048e-01,0.000000
50%,8.128550e-02,4.387273e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2019.000000,7.000000,14.000000,0.000000e+00,0.000000,-1.184433e-01,70.000000,-1.455855e-01,3.393048e-01,0.000000
75%,6.706111e-01,4.792430e-01,0.038000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2020.000000,10.000000,17.000000,6.533771e-01,0.010000,6.185635e-01,80.000000,6.607926e-01,3.393048e-01,0.000000
max,1.843612e+00,1.993750e+00,6.820000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2020.000000,12.000000,23.000000,4.164486e+00,0.490000,4.753296e+00,100.000000,4.584132e+00,3.393048e-01,1.000000


In [65]:
df['Humidity'] = df['Humidity(%)'] / 100

In [66]:
df.describe()

,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,...,Month,Hour,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,Humidity
count,6.293000e+03,6.293000e+03,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,6293.000000,...,6293.000000,6293.000000,6.293000e+03,6293.000000,6.293000e+03,6293.000000,6.293000e+03,6.293000e+03,6293.000000,6293.000000
mean,-3.349137e-14,1.807667e-13,0.134155,0.083426,0.000477,0.242174,0.000159,0.026855,0.098522,0.037343,...,6.759733,12.846178,5.780993e-16,0.013835,-7.000421e-16,68.119021,4.516401e-17,-1.117809e-16,0.000318,0.681190
std,1.000000e+00,1.000000e+00,0.395302,0.276547,0.021830,0.428433,0.012606,0.161673,0.298043,0.189616,...,3.574934,5.813688,1.000000e+00,0.035632,1.000000e+00,16.278361,1.000000e+00,1.000000e+00,0.017826,0.162784
min,-4.789971e+00,-3.693206e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,-1.858416e+00,0.000000,-2.991521e+00,10.000000,-1.665298e+00,-5.644539e+00,0.000000,0.100000
25%,-8.845452e-01,-2.177104e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,8.000000,-8.794668e-01,0.000000,-7.430253e-01,59.000000,-7.658764e-01,3.393048e-01,0.000000,0.590000
50%,8.128550e-02,4.387273e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.000000,14.000000,0.000000e+00,0.000000,-1.184433e-01,70.000000,-1.455855e-01,3.393048e-01,0.000000,0.700000
75%,6.706111e-01,4.792430e-01,0.038000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.000000,17.000000,6.533771e-01,0.010000,6.185635e-01,80.000000,6.607926e-01,3.393048e-01,0.000000,0.800000
max,1.843612e+00,1.993750e+00,6.820000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,12.000000,23.000000,4.164486e+00,0.490000,4.753296e+00,100.000000,4.584132e+00,3.393048e-01,1.000000,1.000000
